In [1]:
%load_ext Cython
%load_ext watermark

In [2]:
%watermark -a "Nelson Liu" -d -n -t -u -v -p numpy,scipy,sklearn,cython,numba -m

Nelson Liu 
last updated: 2016-08-21 20:51:57 

CPython 2.7.12
IPython 5.1.0

numpy 1.11.1
scipy 0.18.0
sklearn 0.17.1
cython 0.24.1
numba 0.27.0

compiler   : GCC 4.2.1 Compatible Apple LLVM 7.3.0 (clang-703.0.31)
system     : Darwin
release    : 15.6.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit


# Optimizing Scientific Python
## + other neat tools to make your life easier!  
Nelson Liu  
August 22, 2016

[download tutorial materials here](https://github.com/nelson-liu/talks_and_tutorials/tree/master/opt_scipy)

> "Premature optimization is the root of all evil"  
>
> ~ Donald Knuth
  
  
Optimized code is more complicated, which leads to it being harder to debug if problems arise!  

Optimizing too early leads to greater development costs further down the road.

# Outline

- Motivating Example / Early Optimization Steps
  - using list comprehensions and NumPy
- Why Python?
- Deeper Optimization
  - "Low" hanging fruit: JIT Compilers
    - Numba and PyPy
  - More complex: C extensions
    - Cython
- Multiprocessing and Multithreading
  - Cython and the GIL

# Motivating Example: Cosine Distance  / Analogies

- Given two vectors, find the angle between them
- Commonly used in semantic similarity tasks; words represented by vectors with a smaller angle of separation are generally "closer" in semantic meaning

- Given vectors $A$ and $B$, the cosine similarity is calculated with the dot product and magnitude.
$$similarity = cos(\theta) = \frac{A \cdot B}{\left|\left|A\right|\right| \left|\left|B\right|\right|}$$

- The analogy prediction task is defined as follows: given a word pair ($w_a, w_b$) (i.e. man, woman) and another
word $w_c$ (i.e. king), predict the best word $w_d$ (i.e. queen) such that the pair ($w_c, w_d$) has
similar relation to ($w_a$, $w_b$).

Namely, to get the solution for an analogy $w_d$:
$$X = vector(w_b) − vector(w_a) + vector(w_c)$$
$$w_d = argmax_{w \in V \forall w \notin \{w_a, w_b, w_c\}} {cos(vector(w), X)} $$


### For didactic purposes, we'll implement an analogy solver in plain Python first, then go about ways to speed it up.

# Let's start by getting some GloVe vectors

In [3]:
try:
   import cPickle as pickle
except:
   import pickle
import numpy as np
import urllib2

# if you have pickled GloVe vectors already, feel free to replace ``None`` with
# a path to a dictionary of word vectors.
# The pickle file I've provided only includes words in the English language as 
# judged by an online dictionary.
local_path = "./data/glove.840B.300d.pkl"
if local_path == None:
    # download pickled word vectors
    pickled_vectors = urllib2.urlopen("http://www.nelsonliu.me/files/glove.840B.300d.pkl")
    glove_vecs = pickle.load(pickled_vectors)
else:
    glove_vecs = pickle.load(open(local_path,"rb"))
    
vocabulary = glove_vecs.keys()

# the dictionary is {word:list}, let's make it {word:ndarray}
# feel free to comment this out if you don't need it
for word in vocabulary:
    glove_vecs[word] = np.array(glove_vecs[word])

# Let's write a preliminary naïve Python implementation


In [4]:
import math

def cosine_sim_py_naive(A, B):
    # calculate the dot product
    dot_prod = 0
    mag_A = 0
    mag_B = 0
    for i in xrange(len(A)):
        dot_prod += A[i]*B[i]
        mag_A += A[i]*A[i]
        mag_B += B[i]*B[i]
    mag_A = math.sqrt(mag_A)
    mag_B = math.sqrt(mag_B)
    
    return dot_prod / (mag_A * mag_B)

# We'll use this method to calculate analogies given `w_a`, `w_b`, `w_c`, and a cosine similarity function.

In [5]:
def calculate_analogies(w_a, w_b, w_c, cosine_sim_func):
    # get the vectors corresponding to the words
    A = glove_vecs[w_a]
    B = glove_vecs[w_b]
    C = glove_vecs[w_c]
    X = np.add(np.subtract(B, A), C)
    
    max_cosine_similarity = -1
    w_d = None
    
    for w_d_candidate in vocabulary:
        if (w_d_candidate == w_a or 
            w_d_candidate == w_b or 
            w_d_candidate == w_c):
            continue
        D_candidate = glove_vecs[w_d_candidate]
        cos_similarity = cosine_sim_func(X, D_candidate)
        if cos_similarity > max_cosine_similarity:
            max_cosine_similarity = cos_similarity
            w_d = w_d_candidate
    return w_d

## Let's time this baseline, wow-I-can't-believe-someone-wrote-this implementation

In [6]:
# this code snippet might take a while to run...
%timeit calculate_analogies("man", "woman", "king", cosine_sim_py_naive)

1 loop, best of 3: 50.9 s per loop


# Wow, that's atrocious! Let's think about some basic ways to optimize it
### For some background: this very task was actually one part of a undergrad NLP homework assignment; we had to solve 10k+ analogies using at least 2 types of pre-trained embeddings. With a runtime like the above for just one analogy, it's no wonder some students spent several days running their code!

# List comprehensions are your friend! 
- Not only do they make your code more concise, they're faster!


In [7]:
def cosine_sim_py_comprehension(A, B):
    # calculate the dot product
    results = [sum(x) for x in zip(*[(i*j, i*i, j*j) for i,j in zip(A, B)])]
    dot_prod = results[0]
    mag_A = math.sqrt(results[1])
    mag_B = math.sqrt(results[2])
    
    return dot_prod / (mag_A * mag_B)

In [8]:
%timeit calculate_analogies("man", "woman", "king", cosine_sim_py_comprehension)

1 loop, best of 3: 31.3 s per loop


That's a pretty sizeable speedup! Now, it'd only take us 81 hours! Jests aside, list comprehensions do offer significant speedup over verbose loops. However, keep in mind that complex list comprehensions can be hard to interpret when you dust off your code 3 months down the line.
- A big reason why this code is so slow with loops is because of Python's dynamic type checking.

- Let's take a look at some other strategies we can use

# Use Numpy Functions

## Why are Numpy Arrays / their functions fast?
- Densely packed, and of homogenous type
  - On the other hand, Python lists are arrays of pointers to objects
  - This gives NumPy the advantage of [locality of reference](https://en.wikipedia.org/wiki/Locality_of_reference)
  
- Most operations are implemented in C
  - Avoids costly dynamic type checking, which really made our previous implementation slow.
  
- Gateway to more optimizations with things like Numba, etc.

# Let's use numpy functions to try to speed this up

In [9]:
def cosine_sim_py_numpy(A, B):
    # calculate the dot product
    dot_prod = np.dot(A,B)
    # calculate the product of magnitudes
    mag_A = np.linalg.norm(A) 
    mag_B = np.linalg.norm(B)
    
    return dot_prod / (mag_A * mag_B)

In [10]:
%timeit calculate_analogies("man", "woman", "king", cosine_sim_py_numpy)

1 loop, best of 3: 1.56 s per loop


Numpy functions gave us a great speedup, as the `dot_prod` and `lingalg.norm` methods use [broadcasting](http://docs.scipy.org/doc/numpy/user/basics.broadcasting.html) to loop over the data structure at the C level.

Lets try some libraries that directly implement cosine similarity or similar routines, and compare their performance.

# Scipy Cosine Distance-Based Implementation

In [11]:
from scipy import spatial

def cosine_sim_py_scipy(A, B):
    return 1 - spatial.distance.cosine(A, B)

In [12]:
%timeit calculate_analogies("man", "woman", "king", cosine_sim_py_scipy)

1 loop, best of 3: 5.42 s per loop


# scikit-learn Cosine Similarity Implementation

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_sim_py_sklearn(A, B):
    return cosine_similarity(A.reshape(1,-1), B.reshape(1,-1))[0][0]

In [14]:
# this is actually surprisingly bad, i've taken it upon myself to see why this is happening
%timeit calculate_analogies("man", "woman", "king", cosine_sim_py_sklearn)

1 loop, best of 3: 22.1 s per loop


# Checkpoint
It seems like our custom method using numpy is the fastest; this makes sense, since the implementations in scikit-learn and scipy have to cater to move than just `ndarray`s and thus spend some time doing validation / other checks.

At this point, we've reached the point that most developers / researchers would get to. It's likely that at this point, we'd just run the our analogy solver and go sleep / do other things for a few hours or days.

However, we can do much better than our current performance by tapping into some other external tools.

# Intermezzo: Why bother using Python in the first place?

If all you care about is performance, you should not be using Python; bare-metal languages like C / C++ are probably more suited to that singular need.  

However, rarely do we only care about performance. Development speed, maintainability, useability, and scalability are all important considerations.

- Python (or mostly-Python) code is easier to read, maintain, and contribute to!
  - This is especially important for replicability

- Python-based tools are easy to run anywhere
    - (Generally) No complicated install or build process required, just setup a {virtual|conda}env, pip install the things you need, and off you go!

# JIT Compilers -- minimal effort, potentially lots of reward


- Good first place to start if you don't want to work a lot (so, everyone)
  - Requires minimal change to your code.


- Two main methods in the Python:
  - PyPy: fast, compliant, alternate implementation of the Python language (2.7, 3.5)
  - Numba: NumPy aware dynamic Python compiler using LLVM

# PyPy, in short

> "If you want your code to run faster, you should probably just use PyPy."  
>
> ~ Guido van Rossum (creator of Python)

- Essentially Python, but with a JIT compiler. 
  - Passes the CPython (the standard implementation of Python) test suite

Unfortunately, it's not fully compatible with NumPy yet, which makes it of limited use to us. It's quite interseting though, and may be game-changing when the SciPy stack is supported.

# Numba

> Numba is a mechanism for producing machine code from Python syntax and typed data structures such as those that exist in NumPy.
>
> ~ [Numba Repo](https://github.com/numba/numba)

- Requires minimal modification to code
  - just add a `jit` decorator to the methods you want to compile

# Installation / Setup

- Numba uses LLVM, a compilation framework
  - which means you need LLVM to run numba, which prevents you from simply doing `pip install numba` in most cases.
  - I hear it works quite well with `conda` though, if you use it.

### Installation on OS X with `brew`

```
brew install homebrew/versions/llvm38 --with-rtti
git clone https://github.com/numba/llvmlite
cd llvmlite
LLVM_CONFIG=/usr/local/Cellar/llvm38/3.8.0/bin/llvm-config-3.8 python setup.py install
LLVM_CONFIG=/usr/local/Cellar/llvm38/3.8.0/bin/llvm-config-3.8 pip install numba
cd ..
rm -rf llvmlite
```

### Installation on Linux

- The instructions below assume you have `clang` and thus `llvm` on your machine
  - If you don't, see if you can ask a system admin to install it `llvm`.
  - Alternatively, you can build `clang` (`llvm` is a dep of clang) in your local directory
    - This sounds like a horrible experience, though. If you do end up doing such a thing, please let me know!

```
git clone https://github.com/numba/llvmlite
cd llvmlite
LLVM_CONFIG=<llvm config file> python setup.py install
LLVM_CONFIG=<llvm config file> pip install numba
cd ..
rm -rf llvmlite
```

more info [here](http://stackoverflow.com/questions/28782512/getting-python-numba-working-on-ubuntu-14-10-or-fedora-21-with-python-2-7), I haven't tried a linux install yet so please let me know if you get one to work!

# Let's try out Numba on our code above

In [15]:
from numba.decorators import jit

cosine_sim_py_numba = jit(cosine_sim_py_naive)

In [16]:
%timeit calculate_analogies("man", "woman", "king", cosine_sim_py_numba)

1 loop, best of 3: 280 ms per loop


## A simple Numba wrapper sped up our original code by ~175x! This is even ~5.5x faster than the function with Numpy.
## With Numba, one could expect to see even greater increases in speed if you have a bunch of nested for loops or the like, since they'd all get turned into machine code

## It's worth noting that all the code we've written so far is still pure Python
## Our Numba function is probably very close to the extent you can push this function without mathematical / algorithmic tricks.

# Some Numba Tips

- It's important to think about what you're optimizing. For example, notice that we chose to optimize our naive Python implementation. Why not optimize our fast numpy implementation, to make it even faster?

In [17]:
cosine_sim_py_numpy_numba = jit(cosine_sim_py_numpy)

In [18]:
%timeit calculate_analogies("man", "woman", "king", cosine_sim_py_numpy_numba)

1 loop, best of 3: 2.12 s per loop


As you can see, running the NumPy implementation with Numba gave us no performance boosts; in fact, our code got a bit slower!
  - If you think about it though, this makes perfect sense
  - The numpy operations internally already use C functions, so the JIT does not help it at all
    - In extreme cases (e.g. this one), the small performance cost added by using Numba is even greater than the optimizations, because the original code is already compiled.

- just calling `jit` directly on the callable generally leads to pretty good results, but there are cases where slowdowns may incur because the C code is forced to fall back on a Python object or the like. 
  - The less Python objects you use, the more `jit` can do for you!
  - numpy arrays are an exception to this rule, because we'll see later that it's quite simple to use them in C.

# C/C++ Extensions to Python

- It's possible to write C code that can be imported to Python as a module, which is quite useful from an efficiency standpoint.
  - Python calls these "extensions" or "extension modules"

- This is what they (roughly) look like

Python Code

```
import some_c_module
result = some_c_module.some_method()
```

C Code
```
static PyObject * some_c_module(PyObject *self)
{
    // some method
    return Py_BuildValue("i", result);
}
```

- This is is generally pretty painful to do, and I wouldn't advise writing C code for use in Python in this way. However, making your own C extensions can be quite useful if you have pre-written code in C and want to have a Python wrapper. There's a pretty good tutorial on doing that [here](http://dan.iel.fm/posts/python-c-extensions/).

- In most cases, you won't have prewritten C code to use.
  - But you still want to optimize your code!
  - But you don't want to dive down the rabbit hole of C extensions / the Python-C API!

# Cython is here for you!

>Cython is an optimising static compiler for both the Python programming language and the extended Cython programming language (based on Pyrex). It makes writing C extensions for Python as easy as Python itself.
>
> [Cython documentation](http://cython.org/)

- It's easy to see that Cython is quite different than Numba. 
  - For one, Cython requires a separate compilation step before running your Python code.

## Let's try writing porting our code to Cython

- I'll use the opportunity to demonstrate everything I wish I knew about Cython when I was first starting out.

# Cython Implementation

In [19]:
%%cython

from __future__ import division
# "cimport" is used to import special compile-time information about the numpy module
cimport numpy as np
cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False) # turn off wrap around for entire function
def cosine_sim_cython(double[:] A, double[:] B):
    cdef size_t i
    cdef double dot_product = 0.0
    cdef double mag_A = 0.0
    cdef double mag_B = 0.0

    # let's rewrite the dot product without numpy
    # and calculate the magnitude in the same loop
    for i in range(A.shape[0]):
        dot_product += A[i] * B[i]
        mag_A += A[i] * A[i]
        mag_B += B[i] * B[i]
    mag_A = sqrt(mag_A)
    mag_b = sqrt(mag_B)
    return dot_product / (mag_A * mag_B)

In [20]:
%timeit calculate_analogies("man", "woman", "king", cosine_sim_cython)

1 loop, best of 3: 448 ms per loop


# Note that the performance of our preliminary Cython implementation was slower than Numba

## and we did a lot more work too!

- However, Cython is powerful because you can choose how low-level you want your code to be. The code above, while it is still Cython, makes use of Python objects which slows down the performance. What if we completely turned off Python?
  - One way of doing this is by removing the GIL. This also gives us the benefit of having easily parallelizable code.

# What's the Global Interpreter Lock (GIL)?

- the GIL is a mutex that prevents multiple native threads from executing Python bytecodes at once
  - In English, it's a "lock" that prevents a Python program from executing multiple threads at the same time.
  - This is necessary because Python's memory is not thread safe!
  - This prevents efficient multi-threading in Python.
      - People use multiprocessing to get around this, but spawning processes is more expensive than spawning threads / they have different memory pools and have to pass objects between each other

- In Cython, you can remove the GIL to easily run your code at the C-level with multithreading.

# Removing the GIL

- In order for code to run with `nogil`, it must satisfy several constraints
  - Only uses statically typed variables of C primitives (e.g. int, long, double, size_t)
  - Arrays must be represented using pointers (goodbye, numpy arrays!)
  - No Python objects or Python methods can be used at all
  - All functions must have `nogil` at the end of their definition.

# But wait, how are we going to use our data without numpy arrays?

- Fortunately, you can easily extract a pointer to the underlying numpy array and it's dtype
```
cdef dtype* X_pointer = <dtype*> X_numpyarray.data
```
- Since we're using doubles here, we have to set the type as such:
```
cdef double* X_pointer = <double*> X_numpyarray.data
```
- Lastly, the original numpy array must be cast as such, explicitly
```
cdef double* X_pointer = <double*> (<numpy.ndarray> X_numpyarray).data
```

# An alternate solution: `MemoryViews`

- typed `MemoryViews` allow you to efficiently access data buffers (e.g. those underlying numpy arrays) without Python overhead
  - `MemoryView` array of doubles : `cdef double [:] <identifier>`
  - `MemoryView` 2d array of ints: `cdef int [:, :] <identifier>`
  - The Cython userguide has [a great page](http://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html) explaining and showing examples of using `MemoryViews`

# Let's write a Cython `nogil`-compatible version of our analogy solver
- We have to be a bit creative with our data because strings are Python objects, and thus not allowed in `nogil`!

In [21]:
def calculate_analogies_cython(w_a, w_b, w_c):
    # get the vectors corresponding to the words
    A = glove_vecs[w_a]
    B = glove_vecs[w_b]
    C = glove_vecs[w_c]

    nd_vectors = np.array(glove_vecs.values())
    return glove_vecs.keys()[calculate_analogies_cython_helper(w_a, w_b, w_c, A, B, C, nd_vectors)]

# And now for the `nogil` Cython methods

In [22]:
%%cython

from __future__ import division
cimport cython
import numpy as np
cimport numpy as np
from libc.math cimport sqrt
from cython.parallel cimport prange, parallel


@cython.boundscheck(False)
def calculate_analogies_cython_helper(str w_a, str w_b, str w_c,
                                      double [:] A_memview, double [:] B_memview,
                                      double [:] C_memview, double [:,:] vectors_memview):
    # build the X array for comparison
    cdef double[:] X_memview = np.add(np.subtract(B_memview, A_memview), C_memview)

    # hardcoded variable for dimensions, figure it out dynamically if i have time to
    # come back and change it
    cdef size_t dimensions = 300

    # keep track of the max cosine similarity and the index of its associated w_d
    cdef double max_cosine_similarity = -1.0
    cdef size_t w_d_idx = -1

    # temp variable for the word vector we're currently comparing
    cdef double[:] d_candidate
    cdef double[:] similarities
    cdef double d_cos_similarity

    # keep track of the number of vectors
    cdef size_t num_vectors = vectors_memview.shape[0]

    # temp variable for iteration, since we can't dynamically generate them
    # in the loop declaration
    cdef size_t i = 0
    with nogil:
        for i in range(num_vectors):
            if(memview_equals(vectors_memview[i], A_memview, dimensions)
               or memview_equals(vectors_memview[i], B_memview, dimensions)
               or memview_equals(vectors_memview[i], C_memview, dimensions)):
                continue
            d_cos_similarity = cosine_sim_cython_nogil(vectors_memview[i], X_memview, dimensions)
            if d_cos_similarity > max_cosine_similarity:
                max_cosine_similarity = d_cos_similarity
                w_d_idx = i

    return w_d_idx

@cython.boundscheck(False)
cdef bint memview_equals(double[:] X, double[:] Y, size_t size) nogil:
    cdef size_t i

    for i in range(size):
        if X[i] != Y[i]:
            return 0
    return 1

@cython.boundscheck(False)
cdef double cosine_sim_cython_nogil(double[:] A, double[:] B, size_t size) nogil:
    cdef size_t i
    cdef double dot_product = 0.0
    cdef double mag_A = 0.0
    cdef double mag_B = 0.0

    for i in prange(size, schedule='guided', num_threads=4):
        dot_product += A[i] * B[i]
        mag_A += A[i] * A[i]
        mag_B += B[i] * B[i]
    mag_A = sqrt(mag_A)
    mag_b = sqrt(mag_B)
    return dot_product / (mag_A * mag_B)


# Phew, that was a lot of work! Let's time it

In [23]:
%timeit calculate_analogies_cython("man", "woman", "king")

1 loop, best of 3: 398 ms per loop


- In terms of speed gains, they were marginal at best
  - This makes a bit of sense, considering that the speed we get from multithreading in this case is mostly offset by the cost of creating threads
- Possible suggestions for increasing speed
  - not checking `MemoryView` equality but comparing Strings instead.
    - How do we get around the fact that strings are objects?
  - At this point, we have pretty good performance for `calculate_analogies`
    - There's little headroom to exploit, so we're much better served (in terms of overall performance) parallelizing the operation of calculating more than one analogy.

# Demo: Building Cython in your project / a look into generated files

# Parting Thoughts



### General Speedup Tips
- Cache constant values in loops and avoid recalculation
  - related: cache values in between loops if you can. Ask if you've calculated something before calculating it again.
- If you're only trying to find the argmax, remove scalars from your equation
- Use numpy! It's pretty good
- Finally, use a JIT or Cythonize your code.

### Thoughts about where to place your optimization efforts
- I had heard about Numba here and there at Scipy, but I never used it until I was making the material for this presentation. Needless to say, I'm extremely impressed by what it can do for single-thread code.
  - One simple modification to our call increased the speed by 175x.
  - It'd be great to see automatic parallelization; they're already [bringing in some constructs to help with that](http://numba.pydata.org/numba-doc/0.11/prange.html)
  - If you use Numba and get good (or bad) results, let me know! I think it could very well be the future of scientific computing.
- However, Cython still has its place for bringing easily-parallelizable loops with OpenMP